In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
python3 = True

# 0- Intruduction: usefull actions

The goal of this work is to build model able to predict probability for a client to leave ENEL (Churn)

## 0.1- Import of useful packages and modules

In [3]:
import pandas as pd, os, sys
pd.set_option('max_colwidth', None)
import numpy as np
from tqdm import tqdm
import numbers
import subprocess
from scipy.stats import chi2_contingency
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr

## 0.2- Environment variables setting

In [4]:
# Setting environment variables
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")
if python3: os.environ['PYSPARK_PYTHON'] = "vsb/CI_conda_env_py3/bin/python"

## 0.2- Spark configuration 

In [5]:
# Importing the Spark Configurator and the Spark Session builder
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as spark_functions
import pyspark.sql.types as types
from pyspark.mllib.stat import Statistics

# Stopping the current session if exists
try:
    session.stop()
except:
    print()
   
# Setting up the spark configurator  
conf = SparkConf()
conf.set('spark.serializer', "org.apache.spark.serializer.KryoSerializer")       
conf.set('spark.executor.memory', '16g')
conf.set('spark.driver.memory', '16g')
conf.set('spark.executor.cores', '3')
conf.set('spark.driver.cores', '3')
conf.set('spark.dynamicAllocation.enabled', 'true')
conf.set('spark.dynamicAllocation.minExecutors', '4')
conf.set('spark.dynamicAllocation.maxExecutors', '8')
conf.set('spark.executor.memoryOverhead', '8096')
conf.set('spark.driver.memoryOverhead', '8096')
conf.set('spark.driver.maxResultSize', '16g')
conf.set('spark.kryoserializer.buffer.max', '2047')
conf.setAppName('DA_Feature_Selection_Step1')
if python3: conf.set("spark.yarn.dist.archives", "/home/anxadvuser/anx_adv_achurn/env_conda_py3.zip#vsb")

# Creating the spark session object
session = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

# 1- Feature Engineering

The first step in the process to build our churn prediction model was feature engineering. In this step we created/merged the data to be used in the modeling step. The following steps were used in the feature Engineering process:
        - Initial timestamps t0 have been choosen. in our case they are 14: 
            -> 2019-05-01, 
            -> 2019-06-01, 
            -> 2019-07-01, 
            -> 2019-08-01,
            -> 2019-09-01,
            -> 2019-10-01, 
            -> 2019-11-01, 
            -> 2019-12-01, 
            -> 2020-01-01, 
            -> 2020-02-01, 
            -> 2020-03-01, 
            -> 2020-04-01, 
            -> 2020-05-01, 
            -> 2020-06-01.
     - A picture of the data has been taken at each instant t0.
     - And finally, 767 features have been computed for each t0 using the data pictures
     - We can immagine the feature computation step as an aggregator merging data from several ENEL systems ("Fatturato", "CBA", "CRM", ...)

# 2- Feature selection

## 2.1- Viewing the data

Reading the data from the hdfs

In [5]:
#input_data = session.read.parquet("/bda/anx_adv_achurn/t_bda_anx_dt_ach_b2b_master_table_train")
input_data = session.read.parquet("/bda/anx_adv_achurn/t_bda_anx_dt_ach_b2b_master_table_train_test")

How many observations do we have in the data ?

In [6]:
input_data.count()

26209006

We have a lot of data and it would be a great thing if the quality of the data was also good. That is wat were are going to check.

How many columns do we have in the data ?

In [11]:
len(input_data.columns)

767

We have 767 columns in the data witch is a quite big number. let's explore the columns to assess whether or not they have an impact on the target variable.

Now let's have a quick look into the data.

In [10]:
input_data_as_pandas_df = input_data.limit(10).toPandas()

In [11]:
input_data_as_pandas_df.head(5)

,idn_servizio_sk,idn_cliente_sk,cdc_stato_servizio_calc,fk_cliente_unico,idc_id_key_servizio,idn_sito_sk,lds_prodotto_listino_2,sds_segmento_new,idc_id_key_cliente,lds_nome_listino,...,qta_days_incasso_fattura_today_servizio_ultima_bolletta_emessa_canale_ALL,qta_days_scadenza_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL,fln_fattura_non_pagata_servizio_ultima_bolletta_emessa_canale_ALL,fln_fattura_pagata_non_totalmente_servizio_ultima_bolletta_emessa_canale_ALL,fln_fattura_pagata_totalmente_in_ritardo_servizio_ultima_bolletta_emessa_canale_ALL,fln_fattura_pagata_totalmente_nei_tempi_servizio_ultima_bolletta_emessa_canale_ALL,val_fattura_non_pagato_servizio_ultima_bolletta_emessa_canale_ALL,val_fattura_servizio_ultima_bolletta_emessa_canale_ALL,val_incasso_servizio_ultima_bolletta_emessa_canale_ALL,idn_run
0,1215,12416792,ATTIVATO,7917813,02i0Y000000kwKCQAY,24649573,ENERGIA PURA CASA,MICRO,0010Y00000G21JGQAZ,Energia Pura Casa,...,137,-6,0,0,0,1,0.00,70.13,70.13,202007040651
1,1215,12416792,ATTIVATO,7917813,02i0Y000000kwKCQAY,24649573,ENERGIA PURA CASA,MICRO,0010Y00000G21JGQAZ,Energia Pura Casa,...,137,-6,0,0,0,1,0.00,70.13,70.13,202007040651
2,4027,16189860,ATTIVATO,12510702,02i0Y000000kx3YQAQ,23786236,ANNO SICURO,MICRO,0010Y00000F6aJ7QAJ,Anno Sicuro P.A. Convenzione,...,358140,358097,1,1,0,0,184.24,184.24,0.00,202007040651
3,4027,16189860,ATTIVATO,12510702,02i0Y000000kx3YQAQ,23786236,ANNO SICURO,MICRO,0010Y00000F6aJ7QAJ,Anno Sicuro P.A. Convenzione,...,358140,358097,1,1,0,0,184.24,184.24,0.00,202007040651
4,8831,16421724,ATTIVATO,8982021,02i0Y000000kyJ2QAI,24234309,ANNO SICURO,MICRO,0010Y00000FCa3LQAT,Anno Sicuro_V3_rinnovi,...,61,8,0,0,1,0,0.00,91.59,91.59,202007040651


## 2.2 Unsupervised feature selection

The main goal of this step is to select the most suitable features for the model first by performing some simple manual analogy and then computing the correlation among features to further filter them.

We are going to perform some preliminary analysis over the entire set of features in order to understand whether or not they are relevant. 

First of all, let's compute the number of distict values for each column in the data set.

### 2.2.1 Distinct values number

In [12]:
distinct_values_dict = {}
for column in tqdm(input_data.columns):
    distinct_values_dict[column] = input_data.select(column).distinct().count()

100%|██████████| 767/767 [1:58:57<00:00,  8.83s/it]  


In [13]:
distinct_values = [distinct_values_dict[column] for column in input_data.columns]
distinct_values_dict_for_pd = {'column': input_data.columns, 'n_distinct_value': distinct_values}
distinct_values_df = pd.DataFrame.from_dict(distinct_values_dict_for_pd)
distinct_values_df.to_csv('/userdata/durand/distinct_values_number_per_column.csv')
distinct_values_df.head(10)

,column,n_distinct_value
0,idn_servizio_sk,2148588
1,idn_cliente_sk,1626057
2,cdc_stato_servizio_calc,6
3,fk_cliente_unico,1585947
4,idc_id_key_servizio,2148588
5,idn_sito_sk,2064097
6,lds_prodotto_listino_2,220
7,sds_segmento_new,3
8,idc_id_key_cliente,1626055
9,lds_nome_listino,876


### 2.2.2 Data types

Now, let's compute the data type of each column in the data set

In [14]:
dta_type_dict = {}
for column in tqdm(input_data.columns):
    if (column.startswith('idn_') or column.startswith('fk_') or column.startswith('qta_')
       or column.startswith('val_') or column.startswith('fln_')):
        dta_type_dict[column] = 'numerical'
    else:
        if (column.startswith('dta_')):
            dta_type_dict[column] = 'date'
        else:
            dta_type_dict[column] = 'categorical'

100%|██████████| 767/767 [00:00<00:00, 305714.26it/s]


In [15]:
data_types = [dta_type_dict[column] for column in input_data.columns]
data_types_dict_for_pd = {'column': input_data.columns, 'data_type': data_types}
data_types_df = pd.DataFrame.from_dict(data_types_dict_for_pd)
data_types_df.to_csv('/userdata/durand/data_types_per_column.csv')
data_types_df.head(10)

,column,data_type
0,idn_servizio_sk,numerical
1,idn_cliente_sk,numerical
2,cdc_stato_servizio_calc,categorical
3,fk_cliente_unico,numerical
4,idc_id_key_servizio,categorical
5,idn_sito_sk,numerical
6,lds_prodotto_listino_2,categorical
7,sds_segmento_new,categorical
8,idc_id_key_cliente,categorical
9,lds_nome_listino,categorical


### 2.2.3 NA proportions

Let's compute the number of NA values for each column in the data set. 

Each column in the data set will have a specific NA calculation method based on the data type. The prefixes of the columns provide us some useful information about the NA calculation method to be used for that particular column. 

let us have a look at all the possible prefixes and establish the NA calculation formulas:

In [16]:
prefixes = list(set([column.split('_')[0] for column in input_data.columns]))
prefixes

['lds',
 'fln',
 'val',
 'sds',
 'dta',
 'cdc',
 'idn',
 'cdn',
 'xds',
 'qta',
 'mds',
 'flc',
 'fk',
 'idc']

'lds' ->
    - Long Description. 
    - Possible values: String
    - NA values: ['***', 'NULL', null, NA, na, Na, nA, '']

'fln' -> 
    - Numerical Flag. 
    - Possible values: [0, 1]
    - NA values: [!=0 and !=1]

'val' ->
    - Numerical value.
    - Possible values: Number
    - NA values: [null]
    
'sds' ->
    - Short Description.
    - Possible values: String
    - NA values: ['***', 'NULL', null, NA, na, Na, nA, '']

'dta' ->
    - Date.
    - Possible values: Date
    - NA values: ['2999-12-01 00:00:00', '2999-12-31 00:00:00', '1900-01-01 00:00:00', null]

'cdc' ->
    - Textual code.
    - Possible values: String
    - NA values: ['***', 'NULL', null, NA, na, Na, nA, '']
    
'idn' -> 
    - Numerical id.
    - Possible values: Number
    - NA values: [0, null]
    

'cdn' ->
    - Numerical code.
    - Possible values: Number
    - NA values: [0, null]
 
'xds' ->
    - Extended Description.
    - Possible values: String
    - NA values: ['***', 'NULL', null, NA, na, Na, nA, '']
  
'qta' ->
    - Numerical value.
    - Possible values: Number
    - NA values: [null]
    
'mds' ->
    - Medium Description.
    - Possible values: String
    - NA values: ['***', 'NULL', null, NA, na, Na, nA, '']

'flc' ->
    - Categorical Flag. 
    - Possible values: Character
    - NA values: ['*', '***', 'NULL', null, NA, na, Na, nA, '']

'fk' ->
    - Numerical value.
    - Possible values: Number
    - NA values: [null, 0]
    
'idc' ->
    - Textual id.
    - Possible values: String
    - NA values: ['***', null, '']

In [17]:
na_values_dict = {}
for column in tqdm(input_data.columns):
    if (column.split('_')[0] == 'lds'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'fln'):
        # NA values: [not a number]
        na_values_dict[column] = input_data.select(column).where("cast(`"+column+"` as double) is null").count()
    if (column.split('_')[0] == 'val'):
        # NA values: [null]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null").count()
    if (column.split('_')[0] == 'sds'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'dta'):
        # NA values: ['2999-12-01 00:00:00', '2999-12-31 00:00:00', '1900-01-01 00:00:00', null]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('2999-12-01 00:00:00', '2999-12-31 00:00:00', '1900-01-01 00:00:00')").count()
    if (column.split('_')[0] == 'cdc'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'idn'):
        # NA values: [null, 0]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in(0)").count()
    if (column.split('_')[0] == 'cdn'):
        # NA values: [null, 0]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in(0)").count()
    if (column.split('_')[0] == 'xds'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'qta'):
        # NA values: [null]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null").count()
    if (column.split('_')[0] == 'mds'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'flc'):
        # NA values: ['*', ***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('*', '***', 'null', 'na', '')").count()
    if (column.split('_')[0] == 'fk'):
        # NA values: [null, 0]
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in(0)").count()
    if (column.split('_')[0] == 'idc'):
        # NA values: ['***', 'null', null, 'na', '']
        na_values_dict[column] = input_data.select(column).where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()

100%|██████████| 767/767 [1:48:51<00:00,  8.08s/it]


In [18]:
na_values = [na_values_dict[column] for column in input_data.columns]
na_values_dict_for_pd = {'column': input_data.columns, 'na_values': na_values}
na_values_df = pd.DataFrame.from_dict(na_values_dict_for_pd)
na_values_df.to_csv('/userdata/durand/na_values_per_column.csv')
na_values_df.head(10)

,column,na_values
0,idn_servizio_sk,0
1,idn_cliente_sk,0
2,cdc_stato_servizio_calc,11667
3,fk_cliente_unico,60
4,idc_id_key_servizio,0
5,idn_sito_sk,0
6,lds_prodotto_listino_2,8011
7,sds_segmento_new,0
8,idc_id_key_cliente,60
9,lds_nome_listino,20337


After analysis we noticed that all the features representing quantities of days, some values were really high. Those high values were due to the fact that a difference was being computed between current date and default date (01-01-1900). Resulting in a very high value. For this reason, all the values above 10 years (3650 days) will be considered as NA for these particular features.

Let's recompute the total number of NA values for these column

In [48]:
date_qta_columns = [
    'qta_giorni_ultimo_contatto_inbound',
    'qta_giorni_ultimo_contatto_outbound',
    'qta_giorni_ultimo_reclamo',
    'qta_days_since_last_inbound_contact',
    'qta_days_since_last_inbound_contact_channel_APP',
    'qta_days_since_last_inbound_contact_channel_FACE',
    'qta_days_since_last_inbound_contact_channel_TELEFONICO',
    'qta_days_since_last_inbound_contact_channel_DOCUMENTALE_COMPLESSO',
    'qta_days_since_last_inbound_contact_channel_DOCUMENTALE',
    'qta_days_since_last_inbound_contact_channel_CHAT',
    'qta_days_since_last_inbound_contact_channel_WEB',
    'qta_days_since_last_inbound_contact_microcausale_RETTIFICHE_CONSUMI_FATTURAZIONE',
    'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA',
    'qta_days_since_last_inbound_contact_microcausale_VOLTURA',
    'qta_days_since_last_inbound_contact_microcausale_PRIMA_ATTIVAZIONE',
    'qta_days_since_last_inbound_contact_microcausale_COPIA_DOCUMENTAZIONI',
    'qta_days_since_last_inbound_contact_microcausale_RIMBORSO_COMPENSAZIONE',
    'qta_days_since_last_inbound_contact_microcausale_LAVORI_E_MODIFICHE_TECNICHE_POTENZA_TENSIONE',
    'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_CREDITO',
    'qta_days_since_last_inbound_contact_microcausale_DISTACCHI_RIALLACCI',
    'qta_days_since_last_inbound_contact_microcausale_DISDETTE_E_CESSAZIONI',
    'qta_days_since_last_inbound_contact_microcausale_GESTIONE_VAS',
    'qta_days_since_last_inbound_contact_microcausale_AUTOLETTURA',
    'qta_days_since_last_inbound_contact_microcausale_RIPENSAMENTO_O_DISCONOSCIMENTO',
    'qta_days_since_last_inbound_contact_microcausale_GESTIONE_FORNITURA',
    'qta_days_since_last_inbound_contact_microcausale_MODIFICA_ANAGRAFICA_O_DATI_FORNITURA_FATTURAZIONE',
    'qta_days_since_last_inbound_contact_microcausale_RECLAMI_SCRITTI',
    'qta_days_since_last_inbound_contact_microcausale_REPEATED_LIST',
    'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_CONSUMI_PAGAMENTI',
    'qta_days_since_last_inbound_contact_microcausale_SWITCH_ATTIVO',
    'qta_days_since_last_inbound_contact_microcausale_MODALITA_DI_PAGAMENTO',
    'qta_days_since_last_inbound_contact_microcausale_ALLACCIO_E_ATTIVAZIONE',
    'qta_days_since_last_inbound_contact_microcausale_VERIFICHE_TECNICHE',
    'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_NON_DI_COMPETENZA',
    'qta_days_since_last_inbound_contact_microcausale_GESTIONE_PRIVACY',
    'qta_days_since_last_inbound_contact_microcausale_RVC',
    'qta_days_since_last_inbound_contact_microcausale_PIANO_DI_RIENTRO',
    'qta_days_since_last_inbound_contact_microcausale_SUBENTRO',
    'qta_days_since_last_inbound_contact_macrocausale_Causale_non_definita',
    'qta_days_since_last_inbound_contact_macrocausale_Chiarimenti_su_fatture',
    'qta_days_since_last_inbound_contact_macrocausale_Chiarimento_su_offerta',
    'qta_days_since_last_inbound_contact_macrocausale_Credito',
    'qta_days_since_last_inbound_contact_macrocausale_Esecuzione_lavori',
    'qta_days_since_last_inbound_contact_macrocausale_Informativa',
    'qta_days_since_last_inbound_contact_macrocausale_Problemi_in_fase_di_attivazione',
    'qta_days_since_last_inbound_contact_macrocausale_Reclami_vendita_Privacy',
    'qta_days_since_last_inbound_contact_macrocausale_Ritardata_esecuzione_subentri_volture_cessazioni_etc',
    'qta_days_last_response_marketing_cli',
    'qta_days_diff_diffida_fattura_cli',
    'qta_days_last_diffida_cli',
    'qta_days_diff_sollecito_fattura_cli',
    'qta_days_last_sollecito_cli',
    'qta_days_servizio_attualmente_distaccato',
    'qta_days_servizio_riallacciato',
    'qta_days_emissione_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL',
    'qta_days_incasso_fattura_today_servizio_ultima_bolletta_emessa_canale_ALL',
    'qta_days_scadenza_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL'
]

In [49]:
date_qta_na_values_dict = {}
for column in tqdm(date_qta_columns):
    # NA values: [null]
    date_qta_na_values_dict[column] = input_data.select(column).where("`"+column+"` is null or "+"`"+column+"` >= 3650").count()

100%|██████████| 54/54 [09:11<00:00, 11.29s/it]


In [50]:
date_qta_na_values = [date_qta_na_values_dict[column] for column in date_qta_columns]
date_qta_na_values_dict_for_pd = {'column': date_qta_columns, 'na_values': date_qta_na_values}
date_qta_na_values_df = pd.DataFrame.from_dict(date_qta_na_values_dict_for_pd)
date_qta_na_values_df.to_csv('/userdata/durand/date_qta_na_values_per_column.csv')
date_qta_na_values_df.head(10)

,column,na_values
0,qta_days_since_last_inbound_contact,9706770
1,qta_days_since_last_inbound_contact_channel_APP,25855692
2,qta_days_since_last_inbound_contact_channel_FACE,20667366
3,qta_days_since_last_inbound_contact_channel_TE...,14276770
4,qta_days_since_last_inbound_contact_channel_DO...,25506166
5,qta_days_since_last_inbound_contact_channel_DO...,19421961
6,qta_days_since_last_inbound_contact_channel_CHAT,25148084
7,qta_days_since_last_inbound_contact_channel_WEB,24331396
8,qta_days_since_last_inbound_contact_microcausa...,25293541
9,qta_days_since_last_inbound_contact_microcausa...,23634710


### 2.2.4 - Filling NA values

#### 2.2.4.1 - Filling NA values: Mode

NA values are filled using the mode over all the observations.

In [7]:
# This function takes in input the df and the column and fill the NA values with the mode
def fill_na_with_mode(data, column, numeric, na_values):
        
    df_mode = data.groupBy(column).count().orderBy('count', ascending = False).limit(1)
    pd_df_mode = df_mode.toPandas()
    mode = list(pd_df_mode[column])[0]
    print ('Column: '+ column)
    print ('Mode: '+ mode)
    print ('Replacing NA values with mode ...')
    
    for na_value in tqdm(na_values):
        if na_value is None:
            data = data.withColumn(column, spark_functions.when(spark_functions.col(column).isNull(), mode).\
            otherwise(spark_functions.col(column)))
        else:
            if numeric:
                data = data.withColumn(column, spark_functions.when(spark_functions.col(column) == na_value, mode).\
                otherwise(spark_functions.col(column)))
            else:
                data = data.withColumn(column, spark_functions.\
                when(spark_functions.lower(spark_functions.col(column)) == str.lower(na_value), mode).\
                otherwise(spark_functions.col(column)))
    return data

In [8]:
column_list_mode = [
    'cdc_stato_servizio_calc',
    'lds_prodotto_listino_2',
    'lds_nome_listino',
    'lds_macro_prodotto',
    'mds_canale_acquisizione',
    'lds_tipo_uso',
    'cdc_id_mercato',
    'cdc_cv_fasce_servizio'
]

In [9]:
#NA number before filling:
for column in column_list_mode:
    print("Number of NA in column "+column+" before filling: "+str(input_data.select(column).\
    where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()))

Number of NA in column cdc_stato_servizio_calc before filling: 11667
Number of NA in column lds_prodotto_listino_2 before filling: 8011
Number of NA in column lds_nome_listino before filling: 20337
Number of NA in column lds_macro_prodotto before filling: 8011
Number of NA in column mds_canale_acquisizione before filling: 31822
Number of NA in column lds_tipo_uso before filling: 2043
Number of NA in column cdc_id_mercato before filling: 838827
Number of NA in column cdc_cv_fasce_servizio before filling: 440144


In [10]:
# Filling the NA values:
for column in tqdm(column_list_mode):
    input_data = fill_na_with_mode(data = input_data, column = column, numeric = False, na_values = [None, '***', 'null', 'na', ''])

  0%|          | 0/5 [00:00<?, ?it/s]

Column: cdc_stato_servizio_calc
Mode: ATTIVATO
Replacing NA values with mode ...



 40%|████      | 2/5 [00:00<00:00, 14.01it/s]

Column: lds_prodotto_listino_2
Mode: ANNO SICURO
Replacing NA values with mode ...



 40%|████      | 2/5 [00:00<00:00, 12.45it/s]

...


 40%|████      | 2/5 [00:00<00:00, 16.40it/s]

Column: cdc_id_mercato
Mode: LIBERO
Replacing NA values with mode ...



 40%|████      | 2/5 [00:00<00:00, 18.02it/s]

Column: cdc_cv_fasce_servizio
Mode: basso
Replacing NA values with mode ...



100%|██████████| 8/8 [01:32<00:00, 11.52s/it]


In [11]:
#NA values after filling:
for column in column_list_mode:
    print("Number of NA in column "+column+" after filling: "+str(input_data.select(column).\
    where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()))

Number of NA in column cdc_stato_servizio_calc after filling: 0
Number of NA in column lds_prodotto_listino_2 after filling: 0
Number of NA in column lds_nome_listino after filling: 0
Number of NA in column lds_macro_prodotto after filling: 0
Number of NA in column mds_canale_acquisizione after filling: 0
Number of NA in column lds_tipo_uso after filling: 0
Number of NA in column cdc_id_mercato after filling: 0
Number of NA in column cdc_cv_fasce_servizio after filling: 0


#### 2.2.4.2 - Filling NA values: 0, ***, -1, OTHER, UNDEFINED, ALTRO 

NA values will be filled using the folowing values: 0, ***, -1, OTHER, UNDEFINED, ALTRO 

In [12]:
column_na = {
'qta_eta_nrm': -1,
'qta_indice_stress_comm_ult_30gg': -1,
'qta_indice_contattosita_ult_30gg': -1,
'val_fascia_stress': -1,
'qta_giorni_ultimo_contatto_inbound': -1,
'qta_giorni_ultimo_contatto_outbound': -1,
'qta_giorni_ultimo_reclamo': -1,
'qta_days_since_last_inbound_contact': -1,
'qta_days_since_last_inbound_contact_channel_APP': -1,
'qta_days_since_last_inbound_contact_channel_FACE': -1,
'qta_days_since_last_inbound_contact_channel_TELEFONICO': -1,
'qta_days_since_last_inbound_contact_channel_DOCUMENTALE_COMPLESSO': -1,
'qta_days_since_last_inbound_contact_channel_DOCUMENTALE': -1,
'qta_days_since_last_inbound_contact_channel_CHAT': -1,
'qta_days_since_last_inbound_contact_channel_WEB': -1,
'qta_days_since_last_inbound_contact_microcausale_RETTIFICHE_CONSUMI_FATTURAZIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA': -1,
'qta_days_since_last_inbound_contact_microcausale_VOLTURA': -1,
'qta_days_since_last_inbound_contact_microcausale_PRIMA_ATTIVAZIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_COPIA_DOCUMENTAZIONI': -1,
'qta_days_since_last_inbound_contact_microcausale_RIMBORSO_COMPENSAZIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_LAVORI_E_MODIFICHE_TECNICHE_POTENZA_TENSIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_CREDITO': -1,
'qta_days_since_last_inbound_contact_microcausale_DISTACCHI_RIALLACCI': -1,
'qta_days_since_last_inbound_contact_microcausale_DISDETTE_E_CESSAZIONI': -1,
'qta_days_since_last_inbound_contact_microcausale_GESTIONE_VAS': -1,
'qta_days_since_last_inbound_contact_microcausale_AUTOLETTURA': -1,
'qta_days_since_last_inbound_contact_microcausale_RIPENSAMENTO_O_DISCONOSCIMENTO': -1,
'qta_days_since_last_inbound_contact_microcausale_GESTIONE_FORNITURA': -1,
'qta_days_since_last_inbound_contact_microcausale_MODIFICA_ANAGRAFICA_O_DATI_FORNITURA_FATTURAZIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_RECLAMI_SCRITTI': -1,
'qta_days_since_last_inbound_contact_microcausale_REPEATED_LIST': -1,
'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_CONSUMI_PAGAMENTI': -1,
'qta_days_since_last_inbound_contact_microcausale_SWITCH_ATTIVO': -1,
'qta_days_since_last_inbound_contact_microcausale_MODALITA_DI_PAGAMENTO': -1,
'qta_days_since_last_inbound_contact_microcausale_ALLACCIO_E_ATTIVAZIONE': -1,
'qta_days_since_last_inbound_contact_microcausale_VERIFICHE_TECNICHE': -1,
'qta_days_since_last_inbound_contact_microcausale_INFORMATIVA_NON_DI_COMPETENZA': -1,
'qta_days_since_last_inbound_contact_microcausale_GESTIONE_PRIVACY': -1,
'qta_days_since_last_inbound_contact_microcausale_RVC': -1,
'qta_days_since_last_inbound_contact_microcausale_PIANO_DI_RIENTRO': -1,
'qta_days_since_last_inbound_contact_microcausale_SUBENTRO': -1,
'qta_days_since_last_inbound_contact_macrocausale_Causale_non_definita': -1,
'qta_days_since_last_inbound_contact_macrocausale_Chiarimenti_su_fatture': -1,
'qta_days_since_last_inbound_contact_macrocausale_Chiarimento_su_offerta': -1,
'qta_days_since_last_inbound_contact_macrocausale_Credito': -1,
'qta_days_since_last_inbound_contact_macrocausale_Esecuzione_lavori': -1,
'qta_days_since_last_inbound_contact_macrocausale_Informativa': -1,
'qta_days_since_last_inbound_contact_macrocausale_Problemi_in_fase_di_attivazione': -1,
'qta_days_since_last_inbound_contact_macrocausale_Reclami_vendita_Privacy': -1,
'qta_days_since_last_inbound_contact_macrocausale_Ritardata_esecuzione_subentri_volture_cessazioni_etc': -1,
'qta_days_last_response_marketing_cli': -1,
'qta_days_emissione_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL': -1,
'qta_days_incasso_fattura_today_servizio_ultima_bolletta_emessa_canale_ALL': -1,
'qta_days_scadenza_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL': -1,
'cdn_cv2_servizio': 0,
'cdn_cv2_cliente': 0,
'val_Altitudine_del_centro': 0,
'val_Superficie_territoriale': 0,
'qta_Grado_di_urbanizzazione': 0,
'cdc_pdr': '***',
'cdc_cap_nrm': '***',
'sds_provincia_fatturazione': '***',
'sds_provincia_fornitura': '***',
'sds_regione_fornitura': '***',
'sds_regione_fatturazione': '***',
'sds_mat_fatturazione': '***',
'sds_mat_fornitura': '***',
'sds_provincia': '***',
'sds_regione': '***',
'sds_mat': '***',
'mds_sottocanale': '***',
'mds_comune': '***',
'cdc_istat': '***',
'cdc_Codice_Regione': '***',
'cdc_Zona_altimetrica': '***',
'cdc_Comune_Montano': '***',
'cdc_causale_attivazione': '***',
'lds_comune_fornitura': '***',
'mds_nome_campagna_last_response_marketing_cli': '***',
'xds_tipo_azione_campagna_last_response_marketing_cli': 'ALTRO',
'xds_tipologia_azione_last_response_marketing_cli': 'ALTRO',
'cdc_cluster_fattura_last': 'OTHER',
'xds_canale_last_response_marketing_cli': 'UNDEFINED'
}

In [13]:
# This function takes in input the df the column and the value to fill with and fill the NA values using the input value
def fill_na_with_value(data, column, numeric, fill_with, na_values):
    print('Filling NA in column ' +column+' with value '+ str(fill_with) + ' ...')
    for na_value in tqdm(na_values):
        if na_value is None:
            data = data.withColumn(column, spark_functions.when(spark_functions.col(column).isNull(), fill_with).\
            otherwise(spark_functions.col(column)))
        else:
            if numeric:
                data = data.withColumn(column, spark_functions.when(spark_functions.col(column) == na_value, fill_with).\
                otherwise(spark_functions.col(column)))
            else:
                data = data.withColumn(column, spark_functions.\
                when(spark_functions.lower(spark_functions.col(column)) == str.lower(na_value), fill_with).\
                otherwise(spark_functions.col(column)))
    
    return data

In [31]:
#NA values before filling:
for column in column_na:
    numeric = isinstance(column_na[column], numbers.Number)
    if (numeric and (column.startswith('qta_days_') or column.startswith('qta_giorni_'))):
        print("Number of NA in column "+column+" before filling: "+str(input_data.select(column).\
        where("`"+column+"` is null or "+"`"+column+"` >= 3650").count()))
    else:
        if(numeric and (not column.startswith('qta_days_')) and  (not column.startswith('qta_giorni_'))):
            print("Number of NA in column "+column+" before filling: "+str(input_data.select(column).\
            where('`'+column+"` is null").count()))
        else:
            print("Number of NA in column "+column+" before filling: "+str(input_data.select(column).\
            where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()))

Number of NA in column xds_tipologia_azione_last_response_marketing_cli before filling: 1373579
Number of NA in column qta_days_since_last_inbound_contact_microcausale_DISDETTE_E_CESSAZIONI before filling: 25454264
Number of NA in column qta_eta_nrm before filling: 35465
Number of NA in column qta_indice_stress_comm_ult_30gg before filling: 8190714
Number of NA in column qta_days_since_last_inbound_contact_microcausale_VERIFICHE_TECNICHE before filling: 25995612
Number of NA in column mds_comune before filling: 1384342
Number of NA in column sds_mat_fatturazione before filling: 12443
Number of NA in column qta_days_since_last_inbound_contact_microcausale_PRIMA_ATTIVAZIONE before filling: 25920734
Number of NA in column qta_days_since_last_inbound_contact_channel_DOCUMENTALE before filling: 19421961
Number of NA in column qta_days_since_last_inbound_contact_macrocausale_Credito before filling: 20260110
Number of NA in column sds_regione_fatturazione before filling: 12443
Number of NA in

In [14]:
for column in tqdm(column_na):
    numeric = isinstance(column_na[column], numbers.Number)
    if numeric:
        input_data = fill_na_with_value(data = input_data, column = column, numeric = True, fill_with = column_na[column], na_values = [None])
    else:
        input_data = fill_na_with_value(data = input_data, column = column, numeric = False, fill_with = column_na[column], na_values = [None, '***', 'null', 'na', ''])

100%|██████████| 1/1 [00:00<00:00, 13.75it/s]

100%|██████████| 1/1 [00:00<00:00, 27.03it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Filling NA in column qta_eta_nrm with value -1 ...
Filling NA in column qta_indice_stress_comm_ult_30gg with value -1 ...
Filling NA in column qta_indice_contattosita_ult_30gg with value -1 ...
Filling NA in column val_fascia_stress with value -1 ...


100%|██████████| 1/1 [00:00<00:00, 25.43it/s]

100%|██████████| 1/1 [00:00<00:00, 25.87it/s]

100%|██████████| 1/1 [00:00<00:00, 25.91it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Filling NA in column qta_giorni_ultimo_contatto_inbound with value -1 ...
Filling NA in column qta_giorni_ultimo_contatto_outbound with value -1 ...
Filling NA in column qta_giorni_ultimo_reclamo with value -1 ...
Filling NA in column qta_days_since_last_inbound_contact with value -1 ...
Filling NA in column qta_days_since_last_inbound_contact_channel_APP with value -1 ...


100%|██████████| 1/1 [00:00<00:00, 25.60it/s]

100%|██████████| 1/1 [00:00<00:00, 26.54it/s]

100%|██████████| 1/1 [00:00<00:00, 26.24it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...


 60%|██████    | 3/5 [00:00<00:00, 20.10it/s]

Filling NA in column cdc_cluster_fattura_last with value OTHER ...



 40%|████      | 2/5 [00:00<00:00, 19.86it/s]

Filling NA in column xds_canale_last_response_marketing_cli with value UNDEFINED ...



100%|██████████| 84/84 [00:08<00:00,  9.47it/s]


For the days quantities, let's treat all values greater than 10 years as NAs and fill them

In [15]:
# This function takes in input the df the column and the value to fill with and fill the NA values using the input value
# It is specific for the day quantity features
def fill_na_with_value_qta(data, column, fill_with):
    print('Filling NA in column ' +column+' with value '+ str(fill_with) + ' ...')
    data = data.withColumn(column, spark_functions.when(spark_functions.col(column) >= 3650, fill_with).\
    otherwise(spark_functions.col(column)))
    return data

In [16]:
for column in tqdm(column_na):
    
    if column_na[column] == -1:
        input_data = fill_na_with_value_qta(data = input_data, column = column, fill_with = column_na[column])

  2%|▏         | 2/84 [00:00<00:04, 16.51it/s]

Filling NA in column qta_eta_nrm with value -1 ...
Filling NA in column qta_indice_stress_comm_ult_30gg with value -1 ...
Filling NA in column qta_indice_contattosita_ult_30gg with value -1 ...
Filling NA in column val_fascia_stress with value -1 ...


  7%|▋         | 6/84 [00:00<00:04, 16.77it/s]

Filling NA in column qta_giorni_ultimo_contatto_inbound with value -1 ...
Filling NA in column qta_giorni_ultimo_contatto_outbound with value -1 ...
Filling NA in column qta_giorni_ultimo_reclamo with value -1 ...
Filling NA in column qta_days_since_last_inbound_contact with value -1 ...


 12%|█▏        | 10/84 [00:00<00:04, 15.78it/s]

...

 60%|█████▉    | 50/84 [00:02<00:02, 16.68it/s]

Filling NA in column qta_days_since_last_inbound_contact_macrocausale_Problemi_in_fase_di_attivazione with value -1 ...
Filling NA in column qta_days_since_last_inbound_contact_macrocausale_Reclami_vendita_Privacy with value -1 ...
Filling NA in column qta_days_since_last_inbound_contact_macrocausale_Ritardata_esecuzione_subentri_volture_cessazioni_etc with value -1 ...


 64%|██████▍   | 54/84 [00:03<00:01, 16.27it/s]

Filling NA in column qta_days_last_response_marketing_cli with value -1 ...
Filling NA in column qta_days_emissione_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL with value -1 ...
Filling NA in column qta_days_incasso_fattura_today_servizio_ultima_bolletta_emessa_canale_ALL with value -1 ...
Filling NA in column qta_days_scadenza_fattura_pagamento_servizio_ultima_bolletta_emessa_canale_ALL with value -1 ...


100%|██████████| 84/84 [00:03<00:00, 26.54it/s]


In [17]:
na_replaced_with_stars = ['cdc_pdr',
'cdc_cap_nrm',
'sds_provincia_fatturazione',
'sds_provincia_fornitura',
'sds_regione_fornitura',
'sds_regione_fatturazione',
'sds_mat_fatturazione',
'sds_mat_fornitura',
'sds_provincia',
'sds_regione',
'sds_mat',
'mds_sottocanale',
'mds_comune',
'cdc_istat',
'cdc_Codice_Regione',
'cdc_Zona_altimetrica',
'cdc_Comune_Montano',
'cdc_causale_attivazione',
'lds_comune_fornitura',
'mds_nome_campagna_last_response_marketing_cli']

In [18]:
#NA values after filling:
for column in column_na:
    numeric = isinstance(column_na[column], numbers.Number)
    if (numeric and (column.startswith('qta_days_') or column.startswith('qta_giorni_'))):
        print("Number of NA in column "+column+" after filling: "+str(input_data.select(column).\
        where("`"+column+"` is null or "+"`"+column+"` >= 3650").count()))
    else:
        if(numeric and (not column.startswith('qta_days_')) and  (not column.startswith('qta_giorni_'))):
            print("Number of NA in column "+column+" after filling: "+str(input_data.select(column).\
            where('`'+column+"` is null").count()))
        else:
            if (column not in na_replaced_with_stars):
                print("Number of NA in column "+column+" after filling: "+str(input_data.select(column).\
                where('`'+column+"` is null or lower("+ column +") in('***', 'null', 'na', '')").count()))
            else:
                print("Number of NA in column "+column+" after filling: "+str(input_data.select(column).\
                where('`'+column+"` is null or lower("+ column +") in('null', 'na', '')").count()))

Number of NA in column qta_eta_nrm after filling: 0
Number of NA in column qta_indice_stress_comm_ult_30gg after filling: 0
Number of NA in column qta_indice_contattosita_ult_30gg after filling: 0
Number of NA in column val_fascia_stress after filling: 0
Number of NA in column qta_giorni_ultimo_contatto_inbound after filling: 0
Number of NA in column qta_giorni_ultimo_contatto_outbound after filling: 0
Number of NA in column qta_giorni_ultimo_reclamo after filling: 0
Number of NA in column qta_days_since_last_inbound_contact after filling: 0
Number of NA in column qta_days_since_last_inbound_contact_channel_APP after filling: 0
Number of NA in column qta_days_since_last_inbound_contact_channel_FACE after filling: 0
Number of NA in column qta_days_since_last_inbound_contact_channel_TELEFONICO after filling: 0
Number of NA in column qta_days_since_last_inbound_contact_channel_DOCUMENTALE_COMPLESSO after filling: 0
Number of NA in column qta_days_since_last_inbound_contact_channel_DOCUMEN

After analysis of the entire set of features given the previously calculated information, the following 115 columns have been tagged as to be removed. Features that will be used to identify the churning entity (idn_servizio_sk, fk_cliente_unico, idc_id_key_servizio, idc_id_key_cliente) will be kepted but will not be used during medeling phase.

In [19]:
column_to_drop_manual_analysis = ['sds_segmento_new', 'idc_id_key_contratto', 
'idn_contratto_sk', 'dta_creazione_att_swp', 'cdc_rid', 'sds_macro_categoria_ateco', 'sds_categoria_ateco_intermedia', 
'val_fascia_contattosita', 'lds_nazione_nascita', 'flc_sesso_nrm', 'flc_vas', 'val_churn', 'val_churn_3_months', 
'val_comunicazione_swp_1_month', 'val_comunicazione_swp_2_months', 'val_comunicazione_swp_3_months', 'mds_provincia', 'mds_land_type', 
'cdc_cd_censimento', 'mds_canale_acqu_fornitura', 'lds_categoria_uso', 'cdc_canale_criticita_infra_last_90_days', 
'cdc_motivazione_criticita_infra_last_90_days', 'cdc_motivazione_criticita_cross_last_90_days', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_LOYALTY_PROMOZIONE_last_3_months', 
'qta_cnt_campagne_mktg_GMC_OUTBOUND_ACQUISITION_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_APP_OUTBOUND_ACQUISITION_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_APP_OUTBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_APP_OUTBOUND_LOYALTY_PROMOZIONE_last_3_months', 
'qta_cnt_campagne_mktg_CS_OUTBOUND_CARING_ALTRO_last_3_months', 'qta_cnt_campagne_mktg_CS_OUTBOUND_UP_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_ACQUISITION_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_ACQUISITION_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_ALTRO_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_ALTRO_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_CARING_ALTRO_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_CARING_CROSSSELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_CARING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_CARING_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_CARING_PROMOZIONE_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_LOYALTY_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_UP_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_DEM_OUTBOUND_UP_SELLING_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_DEM_OUTBOUND_UP_SELLING_TEST_last_3_months', 
'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_ACQUISITION_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_ALTRO_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_CARING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_CARING_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_CROSS_SELLING_PROMOZIONE_last_3_months', 
'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_LOYALTY_PROMOZIONE_last_3_months', 'qta_cnt_campagne_mktg_FATTURA_OUTBOUND_UP_SELLING_PROMOZIONE_last_3_months', 
'qta_cnt_campagne_mktg_RDM_OUTBOUND_RICERCHE_DI_MERCATO_ALTRO_last_3_months', 'qta_cnt_campagne_mktg_S2S_INBOUND_ACQUISITION_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_S2S_INBOUND_CARING_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_S2S_INBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_S2S_INBOUND_CROSS_SELLING_PROMOZIONE_last_3_months', 'qta_cnt_campagne_mktg_S2S_INBOUND_UP_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_1TO1_CROSSSELLING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_SMS_OUTBOUND_ACQUISITION_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_ACQUISITION_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_SMS_OUTBOUND_CARING_CROSSSELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_CARING_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_SMS_OUTBOUND_CARING_INFORMATIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_CARING_PROMOZIONE_last_3_months', 'qta_cnt_campagne_mktg_SMS_OUTBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_LOYALTY_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_SMS_OUTBOUND_UP_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_SMS_OUTBOUND_UP_SELLING_PROMOZIONE_last_3_months', 'qta_cnt_campagne_mktg_TLS_OUTBOUND_1TO1_CROSSSELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_TLS_OUTBOUND_ACQUISITION_DISPOSITIVA_last_3_months', 'qta_cnt_campagne_mktg_TLS_OUTBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 
'qta_cnt_campagne_mktg_WEB_INBOUND_CARING_INFORMATIVA_last_3_months', 'qta_cnt_campagne_mktg_WEB_INBOUND_CROSS_SELLING_DISPOSITIVA_last_3_months', 'lds_RETE_GAS', 
'lds_Cluster_Polis_11_gruppi', 'qta_cnt_mds_tipologia_attivita_gestione_notifiche_m1', 'qta_cnt_mds_tipologia_attivita_gestione_notifiche_m3', 'dta_cessazione', 
'fk_idn_audit_programma', 'dta_attivazione', 'cdc_livello_sollecito_last_diffida_cli', 'sds_descr_canale_altern_soll_elaborato_last_diffida_cli', 
'cdc_procedura_sollecito_last_diffida_cli', 'sds_descr_canale_altern_soll_elaborato_last_sollecito_cli', 'cdc_procedura_sollecito_last_sollecito_cli', 
'qta_days_diff_diffida_fattura_cli', 'qta_days_last_diffida_cli', 'qta_days_diff_sollecito_fattura_cli', 'qta_days_last_sollecito_cli', 
'fln_venditore_mercato_libero_gas_comune_Gruppo_Canarbino', 'fln_venditore_mercato_libero_gas_comune_Unogas', 'fln_venditore_mercato_libero_ele_comune_Unogas', 
'val_Cliente_con_servizio_cessato_last_1_months', 'fln_enelpremia_flag', 'val_punti_avg_m1', 'val_punti_delta_m3', 'val_punti_avg_m3', 'val_punti_min_m3', 
'val_punti_max_m3', 'val_punti_delta_m9', 'val_punti_avg_m9', 'val_punti_min_m9', 'val_punti_max_m9', 'dta_foto_bi', 'cdc_cv_fasce_cliente', 
'qta_days_servizio_attualmente_distaccato', 'qta_days_servizio_riallacciato', 'idn_run', 'cdc_power_gas']

In [21]:
input_data = input_data.drop(*column_to_drop_manual_analysis)

Let's write the output of the NA filling and the columns dropping to the database

In [22]:
input_data.write.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_filled_na")

### 2.2.3 - Creating new features: val_t0_day, val_t0_month and val_t0_year, fln_dual

In [23]:
input_data_filled_na = session.read.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_filled_na")

Let's create the new features.

    - val_t0_day: will not be usefull since all the days are 1st of the month
    - val_t0_month: using dta_t0
    - val_t0_year: using dta_t0
    - fln_dual: using cdc_commodity_attive

In [24]:
input_data_filled_na = input_data_filled_na.withColumn('val_t0_month', spark_functions.month(input_data_filled_na.dta_t0))
input_data_filled_na = input_data_filled_na.withColumn('val_t0_year', spark_functions.year(input_data_filled_na.dta_t0))
input_data_filled_na = input_data_filled_na.withColumn('fln_dual', spark_functions.\
                       when(spark_functions.col('cdc_commodity_attive') == 'DUAL', 1).otherwise(0))

In [25]:
input_data_filled_na.select('dta_t0', 'val_t0_month', 'val_t0_year').limit(3).show()

+-------------------+------------+-----------+
|             dta_t0|val_t0_month|val_t0_year|
+-------------------+------------+-----------+
|2019-09-01 00:00:00|           9|       2019|
|2019-09-01 00:00:00|           9|       2019|
|2019-09-01 00:00:00|           9|       2019|
+-------------------+------------+-----------+



In [26]:
input_data_filled_na.select('cdc_commodity_attive', 'fln_dual').where("cdc_commodity_attive = 'DUAL'").limit(3).show()

+--------------------+--------+
|cdc_commodity_attive|fln_dual|
+--------------------+--------+
|                DUAL|       1|
|                DUAL|       1|
|                DUAL|       1|
+--------------------+--------+



Now let's drop dta_t0 and cdc_commodity_attive

In [27]:
input_data_filled_na = input_data_filled_na.drop('dta_t0')
input_data_filled_na = input_data_filled_na.drop('cdc_commodity_attive')

In [28]:
len(input_data_filled_na.columns)

657

In [29]:
input_data_filled_na.write.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features")

### 2.2.4 - Redundant features

In [6]:
input_data_redundancy_test = session.read.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features")

#### 2.2.4.1 - lds_nome_listino <=> lds_prodotto_listino_2: Chi Square Test

To evaluate the relation between these two features, We are going to implement a Chi square independence test with the help of scipy.stats module.

In [7]:
def chi_square_test(alpha, contingency_pandas_df):
    observed_values = contingency_pandas_df.values    
    stat, p, dof, expected = chi2_contingency(observed_values)
    return {'chi_square_value': stat,'p-value': p, 'degrees_of_freedom': dof}

let's applys this implementation on our two features

In [8]:
df = input_data_redundancy_test.select('lds_nome_listino', 'lds_prodotto_listino_2')
df.show(3)

+--------------------+----------------------+
|    lds_nome_listino|lds_prodotto_listino_2|
+--------------------+----------------------+
|Sempre Con Te Imp...|  SEMPRE CON TE IMP...|
|Sempre Con Te Imp...|     SCEGLI TU 100X100|
|Sempre Con Te Imp...|         SEMPRE CON TE|
+--------------------+----------------------+
only showing top 3 rows



The first thing that we need is a contingency table. This can be obtained by running the following:

In [9]:
contingency_table = df.stat.crosstab('lds_nome_listino', 'lds_prodotto_listino_2')
contingency_table.toPandas().head()

,lds_nome_listino_lds_prodotto_listino_2,1.500 METRI CUBI DI GAS,100X100 GIUSTAXTE,100X100 GIUSTAXTE BIORARIA,100X100 GIUSTAXTE BIORARIA IMPRESA,100X100 GIUSTAXTE GAS,100X100 GIUSTAXTE GAS IMPRESA,100X100 GIUSTAXTE IMPRESA,3.000 METRI CUBI DI GAS,ANNO SICURO,...,TUTELA SIMILE,TUTTOCOMPRESO GAS,TUTTOCOMPRESO LUCE,VALORECASA,VANTAGGIO 5,VANTAGGIO 5+,VANTAGGIO IMPRESA,VANTAGGIO PIU,VANTAGGIO PIU GAS,VANTAGGIO STAGIONE BUS
0,Condominio Flessibile BT Verde rinnovi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Energia Pura V1 28 rinnovi,0,0,0,0,0,0,0,0,17,...,0,0,0,0,0,0,0,0,0,0
2,Soluzione Energia Impresa Business,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Prezzo Amico v1 26 rinnovi_BUS_RI_26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Anno Sicuro P.A._V2_rinnovi,0,0,0,0,0,0,0,0,18223,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Drop the first column lds_nome_listino_lds_prodotto_listino_2 and convert the spark df into a pandas df
contingency_df = contingency_table.drop('lds_nome_listino_lds_prodotto_listino_2')
contingency_pd_df = contingency_df.toPandas()

# Perform the Chi square test
significance_level = 0.05
independenceTestResult = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df)
independenceTestResult

{'chi_square_value': 5233008305.378342,
 'p-value': 0.0,
 'degrees_of_freedom': 190532}

As the p-value is lower than the significance level (0.05), we can confidently reject the null hypothesis and say that there is a dependency between \
lds_nome_listino and lds_prodotto_listino_2.

To decide which of the two feaures we are going to keep, let's check if they are correlated with the target variable (val_churn_2_months)

In [11]:
df = input_data_redundancy_test.select('lds_nome_listino', 'val_churn_2_months')
df1 = input_data_redundancy_test.select('lds_prodotto_listino_2', 'val_churn_2_months')
contingency_table = df.stat.crosstab('lds_nome_listino', 'val_churn_2_months')
contingency_table1 = df1.stat.crosstab('lds_prodotto_listino_2', 'val_churn_2_months')
# Drop the first column and convert the spark df into a pandas df
contingency_df = contingency_table.drop('lds_nome_listino_val_churn_2_months')
contingency_df1 = contingency_table1.drop('lds_prodotto_listino_2_val_churn_2_months')
contingency_pd_df = contingency_df.toPandas()
contingency_pd_df1 = contingency_df1.toPandas()
# Perform the Chi square test
significance_level = 0.05
independenceTestResult = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df)
independenceTestResult1 = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df1)
print('lds_nome_listino vs val_churn_2_months')
print(independenceTestResult)
print('lds_prodotto_listino_2 vs val_churn_2_months')
print(independenceTestResult1)

lds_nome_listino vs val_churn_2_months
{'chi_square_value': 123198.9690061592, 'p-value': 0.0, 'degrees_of_freedom': 874}
lds_prodotto_listino_2 vs val_churn_2_months
{'chi_square_value': 59074.20784380114, 'p-value': 0.0, 'degrees_of_freedom': 218}


As the p-value suggests, both features are correlated with the target. we are going keep the feature with less NA proportion. Namely lds_prodotto_listino_2 (0.03% against 0.07% for lds_nome_listino)

In [12]:
input_data_redundancy_test = input_data_redundancy_test.drop('lds_nome_listino')

#### 2.2.4.2 - lds_macro_prodotto <=> lds_prodotto_listino_2: Chi Square Test

In [13]:
df = input_data_redundancy_test.select('lds_macro_prodotto', 'lds_prodotto_listino_2')
df1 = input_data_redundancy_test.select('lds_macro_prodotto', 'val_churn_2_months')
contingency_table = df.stat.crosstab('lds_macro_prodotto', 'lds_prodotto_listino_2')
contingency_table1 = df1.stat.crosstab('lds_macro_prodotto', 'val_churn_2_months')
# Drop the first column and convert the spark df into a pandas df
contingency_df = contingency_table.drop('lds_macro_prodotto_lds_prodotto_listino_2')
contingency_df1 = contingency_table1.drop('lds_macro_prodotto_val_churn_2_months')
contingency_pd_df = contingency_df.toPandas()
contingency_pd_df1 = contingency_df1.toPandas()
# Perform the Chi square test
significance_level = 0.05
independenceTestResult = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df)
independenceTestResult1 = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df1)
print('lds_macro_prodotto vs lds_prodotto_listino_2')
print(independenceTestResult)
print('lds_macro_prodotto vs val_churn_2_months')
print(independenceTestResult1)

lds_macro_prodotto vs lds_prodotto_listino_2
{'chi_square_value': 5320428218.000001, 'p-value': 0.0, 'degrees_of_freedom': 44254}
lds_macro_prodotto vs val_churn_2_months
{'chi_square_value': 56432.73792303009, 'p-value': 0.0, 'degrees_of_freedom': 203}


As a conclusion there is a dependency between lds_macro_prodotto and lds_prodotto_listino_2 and also between lds_macro_prodotto and the target variable.

We will keep lds_prodotto_listino_2 because of his high number of distinct values (220) with respect to lds_macro_prodotto (205).

In [14]:
input_data_redundancy_test = input_data_redundancy_test.drop('lds_macro_prodotto')

#### 2.2.4.3 - lds_categoria_prodotto <=> lds_prodotto_listino_2: Chi Square Test

In [15]:
df = input_data_redundancy_test.select('lds_categoria_prodotto', 'lds_prodotto_listino_2')
df1 = input_data_redundancy_test.select('lds_categoria_prodotto', 'val_churn_2_months')
contingency_table = df.stat.crosstab('lds_categoria_prodotto', 'lds_prodotto_listino_2')
contingency_table1 = df1.stat.crosstab('lds_categoria_prodotto', 'val_churn_2_months')
# Drop the first column and convert the spark df into a pandas df
contingency_df = contingency_table.drop('lds_categoria_prodotto_lds_prodotto_listino_2')
contingency_df1 = contingency_table1.drop('lds_categoria_prodotto_val_churn_2_months')
contingency_pd_df = contingency_df.toPandas()
contingency_pd_df1 = contingency_df1.toPandas()
# Perform the Chi square test
significance_level = 0.05
independenceTestResult = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df)
independenceTestResult1 = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df1)
print('lds_categoria_prodotto vs lds_prodotto_listino_2')
print(independenceTestResult)
print('lds_categoria_prodotto vs val_churn_2_months')
print(independenceTestResult1)

lds_categoria_prodotto vs lds_prodotto_listino_2
{'chi_square_value': 48175000.11413506, 'p-value': 0.0, 'degrees_of_freedom': 436}
lds_categoria_prodotto vs val_churn_2_months
{'chi_square_value': 894.9776447978869, 'p-value': 4.550652980644098e-195, 'degrees_of_freedom': 2}


As a conclusion there is a dependency between lds_categoria_prodotto and lds_prodotto_listino_2 and also between lds_categoria_prodotto and the target variable.

We will keep lds_prodotto_listino_2 because of his high entropy (220) with respect to lds_categoria_prodotto (3) and also because the p-values suggest that lds_prodotto_listino_2 is more correlated with the target (p-value = 0.0) than lds_categoria_prodotto (p-value = 4.550652980644098e-195)

In [16]:
input_data_redundancy_test = input_data_redundancy_test.drop('lds_categoria_prodotto')

#### 2.2.4.4 - val_fascia_stress <=> qta_indice_stress_comm_ult_30gg: Pearson/Spearman

In [42]:
val_fascia_stress = input_data_redundancy_test.select('val_fascia_stress').toPandas().values
val_fascia_stress = val_fascia_stress.flatten()
qta_indice_stress_comm_ult_30gg = input_data_redundancy_test.select('qta_indice_stress_comm_ult_30gg').toPandas().values
qta_indice_stress_comm_ult_30gg = qta_indice_stress_comm_ult_30gg.flatten()
val_churn_2_months = input_data_redundancy_test.select('val_churn_2_months').toPandas().values
val_churn_2_months = val_churn_2_months.flatten()

In [43]:
# calculate Pearson's correlation
corr, _ = pearsonr(val_fascia_stress, qta_indice_stress_comm_ult_30gg)
print('Pearsons correlation val_fascia_stress vs qta_indice_stress_comm_ult_30gg: %.3f' % corr)

corr, _ = pearsonr(val_fascia_stress, val_churn_2_months)
print('Pearsons correlation val_fascia_stress vs val_churn_2_months: %.3f' % corr)

corr, _ = pearsonr(qta_indice_stress_comm_ult_30gg, val_churn_2_months)
print('Pearsons correlation qta_indice_stress_comm_ult_30gg vs val_churn_2_months: %.3f' % corr)


# calculate spearman's correlation
corr, _ = spearmanr(val_fascia_stress, qta_indice_stress_comm_ult_30gg)
print('Spearmans correlation val_fascia_stress vs qta_indice_stress_comm_ult_30gg: %.3f' % corr)

corr, _ = spearmanr(val_fascia_stress, val_churn_2_months)
print('Spearmans correlation val_fascia_stress vs val_churn_2_months: %.3f' % corr)

corr, _ = spearmanr(qta_indice_stress_comm_ult_30gg, val_churn_2_months)
print('Spearmans correlation qta_indice_stress_comm_ult_30gg vs val_churn_2_months: %.3f' % corr)

Pearsons correlation val_fascia_stress vs qta_indice_stress_comm_ult_30gg: 0.485
Pearsons correlation val_fascia_stress vs val_churn_2_months: -0.019
Pearsons correlation qta_indice_stress_comm_ult_30gg vs val_churn_2_months: -0.007
Spearmans correlation val_fascia_stress vs qta_indice_stress_comm_ult_30gg: 0.769
Spearmans correlation val_fascia_stress vs val_churn_2_months: -0.021
Spearmans correlation qta_indice_stress_comm_ult_30gg vs val_churn_2_months: -0.019


Generally speaking, correlation coefficient values below 0.3 are considered to be weak; 0.3-0.7 are moderate; >0.7 are strong. 
As we can observe above, with Pearson, we obtain a moderate correlation between val_fascia_stress and qta_indice_stress_comm_ult_30gg and a weak correlation of both of them with the target. Whereas with Spearman, we obtain a strong relation betwwen the two features and still a weak relation with the target for both the features.

However, it apears that val_fascia_stress if more correlated with the target than qta_indice_stress_comm_ult_30gg. we will then remove this feature.

In [17]:
input_data_redundancy_test = input_data_redundancy_test.drop('qta_indice_stress_comm_ult_30gg')

In [18]:
len(input_data_redundancy_test.columns)

653

In [19]:
input_data_redundancy_test.write.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features_tmp")
print('Write to new_features_tmp: Done')

#delete parquet before rewriting
print("New_features deletion code:", subprocess.call(["hadoop", "fs", "-rm", "-r", "-skipTrash",\
"/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features"]))

input_data_redundancy_test = session.read.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features_tmp")
input_data_redundancy_test.write.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features")
print('Write to new_features: Done')

#delete parquet before rewriting
print("New_features_tmp deletion code:", subprocess.call(["hadoop", "fs", "-rm", "-r", "-skipTrash",\
"/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features_tmp"]))

Write to new_features_tmp: Done
New_features deletion code: 0
Write to new_features: Done
New_features_tmp deletion code: 0


## 2.3 Supervised feature selection

### 2.3.1 - Categorical features vs Target variable: Chi Squared Test

In [20]:
# Let's fetch the output of the unsupervised feature selection as input of the current step
input_data_supervised = session.read.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_new_features")

In [21]:
# This is the list of the 43 categorical features to be handled
categorical_features = ['cdc_stato_servizio_calc', 'lds_prodotto_listino_2', 'cdc_pdr', 'sds_commodity', 'mds_canale_acquisizione', 'cdc_cap_nrm', 'lds_tipo_uso', 
'cdc_residente', 'mds_tipologia_pagamento', 'cdc_id_mercato', 'sds_provincia_fatturazione', 'sds_provincia_fornitura', 
'sds_regione_fornitura', 'sds_regione_fatturazione', 'sds_mat_fatturazione', 'sds_mat_fornitura', 'sds_provincia', 'sds_regione', 
'sds_mat', 'lds_societa_vendita', 'cdc_win_back_m3', 'mds_sottocanale', 'mds_comune', 'cdc_istat', 'cdc_Codice_Regione', 
'cdc_Zona_altimetrica', 'cdc_Comune_Montano', 'sds_cluster_ele', 'sds_cluster_gas', 'sds_segmento_fasce', 'cdc_causale_attivazione', 
'lds_comune_fornitura', 'cdc_bolletta_web', 'cdc_cluster_fattura_last', 'mds_nome_campagna_last_response_marketing_cli', 
'xds_canale_last_response_marketing_cli', 'xds_tipo_azione_campagna_last_response_marketing_cli', 
'xds_tipologia_azione_last_response_marketing_cli', 'sds_segmento_organizzativo_cliente', 'cdc_power_gas', 'cdc_tenure_bucket', 
'cdc_cv_fasce_servizio', 'cdc_tenure_bucket_cliente']

In [45]:
for column in tqdm(categorical_features):
    df = input_data_supervised.select(column, 'val_churn_2_months')
    contingency_table = df.stat.crosstab(column, 'val_churn_2_months')
    # Drop the first column and convert the spark df into a pandas df
    contingency_df = contingency_table.drop(column+'_val_churn_2_months')
    contingency_pd_df = contingency_df.toPandas()
    # Perform the Chi square test
    significance_level = 0.05
    independenceTestResult = chi_square_test(alpha = significance_level, contingency_pandas_df = contingency_pd_df)
    print(column + ' vs val_churn_2_months')
    print(independenceTestResult)

  2%|▏         | 1/43 [00:06<04:45,  6.80s/it]

cdc_stato_servizio_calc vs val_churn_2_months
{'chi_square_value': 4277006.753767134, 'p-value': 0.0, 'degrees_of_freedom': 4}


  5%|▍         | 2/43 [00:13<04:43,  6.90s/it]

lds_prodotto_listino_2 vs val_churn_2_months
{'chi_square_value': 59074.20784380114, 'p-value': 0.0, 'degrees_of_freedom': 218}


  7%|▋         | 3/43 [00:35<07:28, 11.22s/it]

...

 88%|████████▊ | 38/43 [02:14<00:14,  2.81s/it]

xds_tipologia_azione_last_response_marketing_cli vs val_churn_2_months
{'chi_square_value': 38130.53067669792, 'p-value': 0.0, 'degrees_of_freedom': 12}


 91%|█████████ | 39/43 [02:16<00:14,  3.50s/it]

sds_segmento_organizzativo_cliente vs val_churn_2_months
{'chi_square_value': 4701.400778840139, 'p-value': 0.0, 'degrees_of_freedom': 2}


AnalysisException: "cannot resolve '`cdc_power_gas`' given input columns: [qta_days_since_last_inbound_contact_macrocausale_Reclami_vendita_Privacy, fln_distributore_energia_big, fln_rinnovo_last_1_months_tipologia_ADEGUAMENTO, fln_venditore_mercato_libero_gas_big, val_totale_pagare_x_mean_ciclo_ele_last_3_months, val_consumi_x_max_ciclo_ele_last_6_months, qta_cnt_fln_contatti_inbound_autolet_m12, qta_days_since_last_inbound_contact_microcausale_REPEATED_LIST, val_importo_addebito_x_sum_ciclo_ele_last_6_months, qta_cnt_attivita_pagamenti_metodi_pagamento_last_3_months, val_consumi_x_mean_ciclo_ele_last_1_months, val_importo_addebito_x_min_ciclo_gas_last_1_months, val_importo_addebito_x_max_ciclo_ele_last_3_months, val_pop_res_totale_che_si_sposta_giornalmente_nel_comune_dimora_abituale, qta_days_since_last_inbound_contact_macrocausale_Chiarimenti_su_fatture, val_totale_pagare_x_sum_ciclo_ele_last_3_months, qta_days_since_last_inbound_contact_microcausale_COPIA_DOCUMENTAZIONI, cdc_cap_n

The only variable with p value greater than alpha (0.05) is sds_commodity. this paticular p value shows that there is enough evidence to support that sds_commodity and val_churn_2_months are indipendent. thus sds_commodity can be dropped. 

In [22]:
input_data_supervised = input_data_supervised.drop('sds_commodity')

In [23]:
input_data_supervised.write.parquet("/bda/anx_adv_achurn/durand/t_bda_anx_dt_ach_b2b_master_table_train_test_supervised_fs_step1")